In [2]:
import torch
from PIL import Image
from torch import nn
from torchvision.transforms import transforms

In [3]:
# ResNet网络结构
class BasicBlock(nn.Module): # basic残差块
    expansion = 1 # 扩张大小

    def __init__(self, in_channel, out_channel, stride=1, downsample=None, **kwargs):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample

    def forward(self, x): # 前向传播（残差块）
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x) # 恒等映射

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

class Bottleneck(nn.Module): # 瓶颈残差块
    # width_per_group = 8, groups = 32 分组卷积Resnext架构
    expansion = 4 # 扩张大小

    def __init__(self, in_channel, out_channel, stride=1, downsample=None,
                 groups=1, width_per_group=64):
        super(Bottleneck, self).__init__()

        width = int(out_channel * (width_per_group / 64.)) * groups # 深度

        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=width,
                               kernel_size=1, stride=1, bias=False)  # 压缩通道squeeze channels
        self.bn1 = nn.BatchNorm2d(width)
        # -----------------------------------------
        self.conv2 = nn.Conv2d(in_channels=width, out_channels=width, groups=groups,
                               kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(width)
        # -----------------------------------------
        self.conv3 = nn.Conv2d(in_channels=width, out_channels=out_channel*self.expansion,
                               kernel_size=1, stride=1, bias=False)  # 解压缩通道 unsqueeze channels
        self.bn3 = nn.BatchNorm2d(out_channel*self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x): # 前向传播
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x) # 恒等映射

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self,
                 block,  # 残差块类型
                 blocks_num, # 各类型的残差块数量
                 num_classes=1000, # 分类种类
                 include_top=True, # 是否包含最顶层
                 groups=1, # 分组
                 width_per_group=64): # 分组维度
        super(ResNet, self).__init__()

        self.include_top = include_top
        self.in_channel = 64 # 输入通道数，深度

        self.groups = groups 
        self.width_per_group = width_per_group

        self.conv1 = nn.Conv2d(3, self.in_channel, kernel_size=7, stride=2,
                               padding=3, bias=False)  # 卷积层(1)
        self.bn1 = nn.BatchNorm2d(self.in_channel) # BN层, 用于各通道标准化
        self.relu = nn.ReLU(inplace=True) # 激活函数ReLU
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # 最大池化层
        self.layer1 = self._make_layer(block, 64, blocks_num[0],stride=2) # 残差块（1）,共blocks_num[0]个, 3个
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2) # 残差块（2）,共blocks_num[1]个, 4个
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2) # 残差块（3）,共blocks_num[2]个, 6个
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2) # 残差块（4）,共blocks_num[3]个, 3个
        if self.include_top:
            self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # 自适应平均池化, output size = (1, 1)
            self.fc = nn.Linear(512 * block.expansion, num_classes) # 线性变换

        for m in self.modules():
            if isinstance(m, nn.Conv2d): # 判断该层是否为二维卷积层
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') # 进行权重初始化

    def _make_layer(self, block, channel, block_num, stride=1): # 构造残差块
        downsample = None # 是否下采样
        if stride != 1 or self.in_channel != channel * block.expansion: # 当维度不配时，即残差块与残块之间的连接问题
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion)) # 下采样

        layers = []
        layers.append(block(self.in_channel,
                            channel,
                            downsample=downsample,
                            stride=stride,
                            groups=self.groups,
                            width_per_group=self.width_per_group))
        self.in_channel = channel * block.expansion

        for _ in range(1, block_num):
            layers.append(block(self.in_channel,
                                channel,
                                groups=self.groups,
                                width_per_group=self.width_per_group))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        if self.include_top:
            x = self.avgpool(x)
            x = torch.flatten(x, 1) # 展平变成一维
            x = self.fc(x) # 变成指定维度

        return x


def resnet34(num_classes=1000, include_top=True):
    # https://download.pytorch.org/models/resnet34-333f7ec4.pth
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)



In [8]:
image_path = "./data/test/1.JPG"  # 导入图片
trans = transforms.Compose([transforms.Resize((120, 120)),
                            transforms.ToTensor()])  # 将图片缩放为跟训练集图片的大小一样 方便预测，且将图片转换为张量
image = Image.open(image_path)  # 打开图片
print(image)  # 输出图片 看看图片格式
image = image.convert("RGB")  # 将图片转换为RGB格式
image = trans(image)  # 上述的缩放和转张量操作在这里实现
print(image)  # 查看转换后的样子
image = torch.unsqueeze(image, dim=0)  # 将图片维度扩展一维

classes = ["1", "2", "3", "4", "5", "6", "7",'……']  # 预测种类


<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=381x499 at 0x278E48F5520>
tensor([[[0.1529, 0.1451, 0.1294,  ..., 0.2941, 0.3176, 0.2392],
         [0.1451, 0.1451, 0.1373,  ..., 0.3098, 0.3333, 0.2549],
         [0.1333, 0.1333, 0.1373,  ..., 0.2980, 0.3255, 0.2549],
         ...,
         [0.7255, 0.7373, 0.7294,  ..., 0.7255, 0.6314, 0.6196],
         [0.7020, 0.7294, 0.7451,  ..., 0.6157, 0.5490, 0.5255],
         [0.6745, 0.7176, 0.7333,  ..., 0.6745, 0.5804, 0.4235]],

        [[0.2549, 0.2431, 0.2275,  ..., 0.4078, 0.4431, 0.3725],
         [0.2471, 0.2471, 0.2353,  ..., 0.4000, 0.4353, 0.3647],
         [0.2392, 0.2314, 0.2353,  ..., 0.3961, 0.4275, 0.3686],
         ...,
         [0.7569, 0.7647, 0.7569,  ..., 0.6863, 0.6000, 0.5882],
         [0.7216, 0.7451, 0.7608,  ..., 0.5961, 0.5216, 0.4902],
         [0.6902, 0.7294, 0.7412,  ..., 0.6627, 0.5608, 0.4000]],

        [[0.3843, 0.3922, 0.3882,  ..., 0.8000, 0.8235, 0.7333],
         [0.3765, 0.3922, 0.3922,  ..., 0.

In [6]:
# 预测
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # 将代码放入GPU进行训练
print("using {} device.".format(device))

net = resnet34()
net.load_state_dict(torch.load("Resnet34.pth", map_location=device))
net.to(device)
net.eval()  # 关闭梯度，将模型调整为测试模式
with torch.no_grad():  # 梯度清零
    outputs = net(image.to(device))  # 将图片打入神经网络进行测试
    # print(net)  # 输出模型结构
    print(outputs.shape)  # 输出预测的张量数组
    ans = (outputs.argmax(1)).item()  # 最大的值即为预测结果，找出最大值在数组中的序号，
    # 对应找其在种类中的序号即可然后输出即为其种类
    print(classes[ans])

using cuda:0 device.
torch.Size([1, 1000])
1
